In [2]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    import os

    drive.mount('/content/drive')
    os.chdir('/content/drive/Shareddrives/Confidence Probes/taskdiscovery')
    !pip3 install transformer_lens fancy_einsum einops datasets encoders plotly circuitsvis

else:
    print("Not running in Google Colab, skipping installation of requirements.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.5/594.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/13

In [3]:
from training_utils import load_model_data
from training_utils import LinePlot
from lens import get_lens_loss
import torch
from tqdm import tqdm
import numpy as np
import pickle
import math


## Load model and data
We will be using gpt2 small for now.

In [ ]:
model_name = "gpt2-small"
batch_size = 32
device, model, tokenizer, owt_iter = load_model_data(model_name, batch_size)
n_layers = model.cfg.n_layers
n_heads = model.cfg.n_heads
head_dim = model.cfg.d_head
d_model = model.cfg.d_model
lr = 1e-2

folder = "./results/"
shared_bias = False

print(n_layers, n_heads, head_dim, d_model)

## Create Modal Lens Prameters

In [ ]:
prior_bias = [model.blocks[i].attn.b_O.clone() for i in range(n_layers)]

attn_bias = [
    torch.nn.Parameter(
        (prior_bias[i] if shared_bias else prior_bias[i].repeat(i + 1, 1)).to(device)
    )
    for i in range(n_layers)
]

lens_optimizer = torch.optim.AdamW(attn_bias, lr=lr, weight_decay=0)

for param in model.parameters():
    param.requires_grad = False


## Training Loop

In [ ]:
lp = LinePlot([*[f"kl_loss_{k}" for k in range(n_layers)], "step_size"])

for i in tqdm(range(5000)):
    batch = next(owt_iter)["tokens"]
    lens_optimizer.zero_grad()

    kl_losses, _ = get_lens_loss(batch, model, n_layers, attn_bias,batch_size)
    kl_losses = kl_losses.mean(dim=-1)
    loss = torch.clamp(kl_losses.sum(), max=100)
    loss.backward()

    prev_weights = torch.cat(attn_bias, dim=0).detach()

    lens_optimizer.step()

    step_sz = (torch.cat(attn_bias, dim=0) - prev_weights).abs().sum()
    lp.add_entry(
        {
            "step_size": step_sz.item(),
            **{f"kl_loss_{k}": kl_losses[k].item() for k in range(n_layers)},
        }
    )

    # lens_scheduler.step()

    if math.isnan(lp.stat_book["step_size"][-1]):
        break

    if i % 250 == 0:
        lp.plot(subplots=3, save=f"{folder}/train.png", twinx=False, mv=20)
        with open(f"{folder}/modal_lens_weights.pkl", "wb") as f:
            pickle.dump(attn_bias, f)

  0%|          | 0/5000 [00:00<?, ?it/s]


TypeError: get_lens_loss() missing 4 required positional arguments: 'model', 'n_layers', 'attn_bias', and 'batch_size'